In [46]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
from tqdm import tqdm
import wandb
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from collections import Counter
import random
import torch.nn.functional as F

In [47]:
# Path to the CSV file
test_data_dir = '/kaggle/input/akshara-dataset/aksharantar_sampled/hin/hin_test.csv'
train_data_dir = '/kaggle/input/akshara-dataset/aksharantar_sampled/hin/hin_train.csv'
val_data_dir = '/kaggle/input/akshara-dataset/aksharantar_sampled/hin/hin_valid.csv'

train_data = pd.read_csv(train_data_dir)
val_data = pd.read_csv(val_data_dir)
test_data = pd.read_csv(test_data_dir)


# print(train_data.shape[0])
# print(val_data.shape[0])
# print(test_data.shape[0])
print(train_data.shape[0] + val_data.shape[0] + test_data.shape[0])

59389


In [48]:

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def load_data(train_path, test_path, val_path):
    train_data = pd.read_csv(train_path)

    input_texts = train_data.iloc[:, 0].tolist()
    target_texts = train_data.iloc[:, 1].tolist()
    
    val_input_texts = val_data.iloc[:, 0].tolist()
    val_target_texts = val_data.iloc[:, 1].tolist()
    
    test_input_texts = test_data.iloc[:, 0].tolist()
    test_target_texts = test_data.iloc[:, 1].tolist()

    input_characters = sorted(list(set(''.join(input_texts))))
    target_characters = sorted(list(set(''.join(target_texts))))

    input_token_index = dict([(char, i+3) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i+3) for i, char in enumerate(target_characters)])

    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)

    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])

    encoder_input_data = [[input_token_index[char] for char in txt] for txt in input_texts]
    decoder_input_data = [[target_token_index[char] for char in txt] for txt in target_texts]
    decoder_target_data = [[target_token_index[char] for char in txt[1:]] for txt in target_texts]

    encoder_input_data = pad_sequence([torch.tensor(seq) for seq in encoder_input_data], batch_first=True)
    decoder_input_data = pad_sequence([torch.tensor(seq) for seq in decoder_input_data], batch_first=True)
    decoder_target_data = pad_sequence([torch.tensor(seq) for seq in decoder_target_data], batch_first=True)

    return input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data, val_input_texts, val_target_texts, test_input_texts, test_target_texts
#     return input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data

# Path to the CSV files
input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data, val_input_texts, val_target_texts, test_input_texts, test_target_texts = load_data(train_data_dir, test_data_dir, val_data_dir)

# input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data = load_data(train_data_dir, test_data_dir, val_data_dir)

print(input_token_index)
print(decoder_input_data[0])
print(decoder_input_data.shape)

{'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
tensor([40, 55, 37, 66, 35, 66, 43, 54,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0])
torch.Size([51199, 20])


In [49]:
# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [50]:
# Define the dataset class
class TransliterationDataset(Dataset):
    def __init__(self, input_texts, target_texts, input_token_index, target_token_index):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.input_token_index = input_token_index
        self.target_token_index = target_token_index

        # Add missing characters to target_token_index
        for text in target_texts:
            for char in text:
                if char not in target_token_index:
                    target_token_index[char] = len(target_token_index)

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Convert text sequences to token sequences
        input_seq = [self.input_token_index[char] for char in input_text]
        target_seq = [self.target_token_index[char] for char in target_text]

        return torch.tensor(input_seq), torch.tensor(target_seq)
    

def collate_fn(batch):
    input_seqs, target_seqs = zip(*batch)
    
    # Pad input sequences
    input_seqs = pad_sequence(input_seqs, batch_first=True)

    # Pad target sequences with zeros at the beginning
    max_target_length = max(len(seq) for seq in target_seqs)
    padded_target_seqs = []
    for seq in target_seqs:
        pad_length = max_target_length - len(seq)
        padded_seq = [0] * 2 + seq.tolist()
        padded_target_seqs.append(torch.tensor(padded_seq))

    # Pad all target sequences to the same maximum length
    padded_target_seqs = pad_sequence(padded_target_seqs, batch_first=True)
    
    return input_seqs, padded_target_seqs

In [51]:
'''
Define the Encoder class.
'''
class Encoder(nn.Module):
    '''
    The Encoder class creates the encoder for the sequence-to-sequence model.
    '''

    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout, cell_type, bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        self.embedding_size = embedding_size

        # Embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder recurrent layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)

    def forward(self, x):
        # Embedding layer
        embedded = self.dropout(self.embedding(x))
        
        # RNN layer
        output, hidden = self.rnn(embedded)
        
        # Extracting hidden states for bidirectional RNN
        if isinstance(self.rnn, nn.LSTM):
            hn, cn = hidden
        else:
            hn, cn = hidden, None
        
        # Extracting the last hidden state
        if self.bidirectional:
            if isinstance(self.rnn, nn.LSTM):
                hn, cn = hidden
                return torch.sum(hn[:-2], dim=0, keepdim=True), torch.sum(cn[-2:], dim=0, keepdim=True)
            else:
                return torch.sum(hidden[:-2], dim=0, keepdim=True), None
        else:
            if isinstance(self.rnn, nn.LSTM):
                hn, cn = hidden
                return hn[-1].unsqueeze(0), cn[-1].unsqueeze(0)
            else:
                return hn[-1].unsqueeze(0), None


In [52]:
'''
Define the Decoder class.
'''
class AttentionDecoder(nn.Module):
    '''
    The AttentionDecoder class creates the decoder with attention mechanism for the sequence-to-sequence model.
    '''

    def __init__(self, output_size, embedding_size, hidden_size, encoder_hidden_size, num_layers, dropout, cell_type):
        super(AttentionDecoder, self).__init__()
        
        # Checking if LSTM is selected
        if cell_type=="LSTM":
            raise NotImplementedError("LSTM is not implemented yet. Select either rnn or gru")
        
        # Initializing decoder parameters
        self.hidden_size = hidden_size
        self.encoder_hidden_size = encoder_hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        self.output_size = output_size
        self.embedding_size = embedding_size
        
        # Embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
    
        self.encoder_hn, self.decoder_cn = None, None
        
        # Decoder recurrent layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)
        

    def init_weights(self):
        # Initializing attention weights
        assert (self.encoder_hn is not None)
        self.U = nn.Linear(self.hidden_size, self.hidden_size) # weights for decoder embedding
        self.W = nn.Linear(self.encoder_hn.shape[-1], self.hidden_size) # weights for encoder hidden state
        self.V = nn.Linear(self.hidden_size, 1) # weights for final output
        self.encoder_hn_proj = self.W(self.encoder_hn.transpose(1,0)).to(device)
        
    def forward(self, x, prev_state):
        # Embedding layer
        embedded = self.dropout(self.embedding(x))
        decoder_embed_proj = self.U(prev_state)    
        e = self.V(F.tanh(self.encoder_hn_proj + decoder_embed_proj.unsqueeze(1))).squeeze(-1)
        a = F.softmax(e, dim=1)
        context = torch.matmul(self.encoder_hn_proj.transpose(2,1), a.unsqueeze(2)).squeeze(-1)
        
        context = context.unsqueeze(0).repeat(self.num_layers,1,1)
        
        # RNN layer
        output, hidden = self.rnn(embedded.unsqueeze(1), context)
        
        # Output layer
        output = self.fc(output)
        output = output.squeeze(1)
        
        return output, hidden


In [53]:
class Seq2SeqAttention(nn.Module):
    '''
    The Seq2SeqAttention class creates the sequence-to-sequence model with attention mechanism.
    '''

    def __init__(self, encoder, decoder, device):
        super(Seq2SeqAttention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        source_len = source.size(1)
        target_vocab_size = self.decoder.output_size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)
        
        encoder_hn = torch.zeros(source_len, batch_size, self.encoder.hidden_size)
        encoder_cn = torch.zeros(source_len, batch_size, self.encoder.hidden_size)

        # Encoding the source sequence
        for t in range(source_len):
            encoder_hn_step, encoder_cn_step = self.encoder(source[:, t].unsqueeze(1))
            encoder_hn[t] = encoder_hn_step
            if encoder_cn_step is not None:
                encoder_cn[t] = encoder_cn_step
        
        self.decoder.encoder_hn = encoder_hn
        self.decoder.init_weights()
        self.decoder.to(self.device)
        
        # Decoding the target sequence
        decoder_hn = torch.zeros(batch_size, self.decoder.hidden_size).to(self.device)
        decoder_input = target[:, 0]
        for t in range(0, target_len):
            decoder_output, decoder_hn = self.decoder(decoder_input, decoder_hn)
            decoder_hn = decoder_hn[-1, :, :]
            outputs[:, t, :] = decoder_output
            best_guess = decoder_output.argmax(1)
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = decoder_output.max(1)[1]
            decoder_input = (target[:, t] if teacher_force else top1)

        return outputs


In [54]:
# Define model parameters
cell_type = 'GRU'
n_enc_layers = 1
n_dec_layers = 1
hidden_layer_size = 256
num_encoder_tokens = 100
num_decoder_tokens = 100
dropout = 0.2
emb_size = 16
batch_size = 64
bidirectional=True

# Modify build_model function to accept device argument
def build_model(cell_type, n_enc_layers, n_dec_layers, hidden_layer_size, num_encoder_tokens, num_decoder_tokens, dropout, emb_size, device, bidirectional):
    encoder = Encoder(input_size=num_encoder_tokens, 
                      embedding_size=emb_size, 
                      hidden_size=hidden_layer_size, 
                      num_layers=n_enc_layers, 
                      dropout=dropout, 
                      cell_type=cell_type, 
                      bidirectional=bidirectional).to(device)

    decoder = AttentionDecoder(output_size=num_decoder_tokens, 
                      embedding_size=emb_size, 
                      hidden_size=hidden_layer_size,
                      encoder_hidden_size=hidden_layer_size,
                      num_layers=n_dec_layers, 
                      dropout=dropout, 
                      cell_type=cell_type).to(device)

    model = Seq2SeqAttention(encoder, decoder, device).to(device)
    print(model)
    
    return model

# Load data on GPU
train_dataset = TransliterationDataset(input_texts, target_texts, input_token_index, target_token_index)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# print(train_dataset[0])
# for i ,j in train_loader:
#     print(i,j)
#     break

# Initialize model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = build_model(cell_type, n_enc_layers, n_dec_layers, hidden_layer_size, num_encoder_tokens, num_decoder_tokens, dropout, emb_size, device, bidirectional=True)
optimizer = torch.optim.Adam(model.parameters())

# Training loop
def train(model, optimizer, train_loader, device, criterion, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
            for batch_idx, (input_seqs, target_seqs) in enumerate(tepoch):
                input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
                optimizer.zero_grad()
                output = model(input_seqs, target_seqs)
                loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

                _, predicted = torch.max(output, 2)
                correct_predictions += (predicted == target_seqs).sum().item()
#                 print(f"predicted: {predicted} target_seqs:{target_seqs}")
                total_predictions += target_seqs.numel()
                train_acc = correct_predictions / total_predictions
                tepoch.set_postfix(loss=total_loss/(batch_idx+1), accuracy=train_acc)
                train_loss = total_loss/len(train_loader)
                
#                 correct_predictions += (predicted == target_seqs).all(dim=1).sum().item()
# #                 print(f"predicted: {predicted} target_seqs:{target_seqs}")
#                 total_predictions += target_seqs.size(0)  #numel()
#                 break
                
            print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {correct_predictions/total_predictions:.4f}, Loss: {total_loss/len(train_loader):.4f}")
            print(correct_predictions)
#             wandb.log({
#                 "train_loss": train_loss,
#                 "train_accuracy": train_acc,
#                 "epoches":epoch
#             })
    return model, train_acc, train_loss

            
# Define the loss function
criterion = nn.CrossEntropyLoss()
# print(model)
# Train the model
model1, train_acc, train_loss = train(model, optimizer, train_loader, device, criterion)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2SeqAttention(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 16)
    (rnn): GRU(16, 256, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): AttentionDecoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 16)
    (rnn): GRU(16, 256, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=256, out_features=100, bias=True)
  )
)


Epoch 1/1: 100%|██████████| 800/800 [00:56<00:00, 14.05batch/s, accuracy=0.549, loss=2.21]

Epoch [1/1], Accuracy: 0.5494, Loss: 2.2111
485081


In [55]:
def validate(model1, val_loader, device, criterion):
    model.eval()
    val_total_loss = 0
    val_correct_predictions = 0
    val_total_predictions = 0
    with torch.no_grad():
        for val_input_seqs, val_target_seqs in tqdm(val_loader, desc="Validation"):
            val_input_seqs, val_target_seqs = val_input_seqs.to(device), val_target_seqs.to(device)
            val_output = model(val_input_seqs, val_target_seqs)
            val_loss = criterion(val_output.view(-1, val_output.size(-1)), val_target_seqs.view(-1))
            val_total_loss += val_loss.item()
            
# #             print(f"val_predicted: {val_predicted}  val_target_seqs: {val_target_seqs}")
#             val_correct_predictions += (val_predicted == val_target_seqs).all(dim=1).sum().item()
# #             print(f"val_correct_predictions: {val_correct_predictions}")
#             val_total_predictions += val_target_seqs.size(0)  #numel()

    
            _, val_predicted = torch.max(val_output, 2)
            val_correct_predictions += (val_predicted == val_target_seqs).sum().item()
            val_total_predictions += val_target_seqs.numel()

    val_acc = val_correct_predictions / val_total_predictions
    val_loss = val_total_loss / len(val_loader)

    return val_loss, val_acc

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define your validation dataset
val_dataset = TransliterationDataset(val_input_texts, val_target_texts, input_token_index, target_token_index)

# Define your validation data loader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

num_epochs = 1

# Validation loop
for epoch in range(num_epochs):
    val_loss, val_acc = validate(model, val_loader, device, criterion)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


Validation: 100%|██████████| 64/64 [00:01<00:00, 37.07it/s]

Validation Loss: 1.9444, Validation Accuracy: 0.5969


In [56]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

def decode_indices(indices, latin_idx2token):
    decoded_string = ''
    for idx in indices:
        if idx not in (0, 1, 2):
            decoded_string += latin_idx2token[idx]
    return decoded_string

def predict(model, iterator, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for src, trg in iterator:
            src = src.to(device)
            trg = trg.to(device)
            output = model(src, trg, teacher_forcing_ratio=0.0)  # Turn off teacher forcing
            output = output.argmax(2)
            src = src.cpu().numpy()
            trg = trg.cpu().numpy()
            output = output.cpu().numpy()
            predictions.append((src,trg, output))
    return predictions

# Assuming latin_idx2token and devanagari_idx2token are dictionaries mapping indices to tokens
latin_idx2token = {idx: char for char, idx in input_token_index.items()}
devanagari_idx2token = {idx: char for char, idx in target_token_index.items()}

# Load test data
test_dataset = TransliterationDataset(test_input_texts, test_target_texts, input_token_index, target_token_index)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Make predictions
test_predictions = predict(model, test_loader, device)

# Collect predictions in a list of dictionaries
results = []

for src_indices, trg_indices, output_indices in test_predictions:
    for j in range(src_indices.shape[0]):
        input_text = decode_indices(src_indices[j], latin_idx2token)
        actual_text = decode_indices(trg_indices[j], devanagari_idx2token)
        predicted_text = decode_indices(output_indices[j], devanagari_idx2token)
        results.append({
            'Input Text': input_text,
            'Actual Text': actual_text,
            'Predicted Text': predicted_text
        })

# Convert results to DataFrame
df = pd.DataFrame(results)

# Save DataFrame to CSV with UTF-8 encoding
csv_file_path = 'seq2seq_predictions_with_attn.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print('Predictions saved to seq2seq_predictions_with_attn.csv')

# Read and display the CSV file using UTF-8 encoding
df_loaded = pd.read_csv(csv_file_path, encoding='utf-8')

# Generate download link for the CSV file (for use in Jupyter notebook)
def create_download_link(filename):
    import base64
    from IPython.display import HTML
    with open(filename, 'rb') as f:
        b64 = base64.b64encode(f.read()).decode()
    return HTML(f'<a download="{filename}" href="data:file/csv;base64,{b64}">Download {filename}</a>')

# Display the download link (for use in Jupyter notebook)
create_download_link(csv_file_path)


Predictions saved to seq2seq_predictions_with_attn.csv


In [57]:
wandb.login(key='12c0b23d6865ce943b48c8ea1451c9b2d3aedf60')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [58]:
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',  # Random search
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  # Objective: Maximize accuracy
    },
    'parameters': {
        'cell_type': {'values': ['RNN', 'GRU']},
        'n_enc_layers': {'values': [1, 2, 3]},
        'n_dec_layers': {'values': [1, 2, 3]},
        'hidden_layer_size': {'values': [16, 32, 64, 256]},
        'emb_size': {'values': [16, 32, 64, 256]},
        'bidirectional': {'values': [True,False]},
        'dropout': {'values': [0.2, 0.3]},
        'epochs': {'values': [5, 8, 3]} 
    }
}


# Initialize a new wandb sweep
sweep_id = wandb.sweep(sweep_config, project="DL_Assignment-3")

Create sweep with ID: x9ncp2p6
Sweep URL: https://wandb.ai/lokendrakumar/DL_Assignment-3/sweeps/x9ncp2p6


In [59]:
import torch
import torch.nn as nn

def train_model():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name based on hyperparameters
        run_name = f"{wandb.config.cell_type}_enc-{wandb.config.n_enc_layers}_dec-{wandb.config.n_dec_layers}_hidden-{wandb.config.hidden_layer_size}_emb-{wandb.config.emb_size}_bi-{wandb.config.bidirectional}_drop-{wandb.config.dropout}_epochs-{wandb.config.epochs}"
        wandb.run.name = run_name

        # Initialize the model
        model = build_model(wandb.config.cell_type,
                            wandb.config.n_enc_layers,
                            wandb.config.n_dec_layers,
                            wandb.config.hidden_layer_size,
                            num_encoder_tokens,
                            num_decoder_tokens,
                            wandb.config.dropout,
                            wandb.config.emb_size,
                            device,
                            wandb.config.bidirectional)
        model.to(device)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()  # Use nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Training and validation loop
        for epoch in range(wandb.config.epochs):
            model.train()
            total_loss = 0
            correct_predictions = 0
            total_predictions = 0

            for input_seqs, target_seqs in tqdm(train_loader, desc="Training"):
                input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
                optimizer.zero_grad()
                output = model(input_seqs, target_seqs)
                loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                
                _, predicted = torch.max(output, 2)
                correct_predictions += (predicted == target_seqs).sum().item()
#                 print(f"predicted: {predicted} target_seqs:{target_seqs}")
                total_predictions += target_seqs.numel()

            train_acc = correct_predictions / total_predictions
            train_loss = total_loss / len(train_loader)
            wandb.log({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
            })

            # Validation phase
            model.eval()
            val_total_loss = 0
            val_correct_predictions = 0
            val_total_predictions = 0
            with torch.no_grad():
                for val_input_seqs, val_target_seqs in tqdm(val_loader, desc="Validation"):
                    val_input_seqs, val_target_seqs = val_input_seqs.to(device), val_target_seqs.to(device)
                    val_output = model(val_input_seqs, val_target_seqs)
                    val_loss = criterion(val_output.view(-1, val_output.size(-1)), val_target_seqs.view(-1))
                    val_total_loss += val_loss.item()
                    
                    _, val_predicted = torch.max(val_output, 2)
                    val_correct_predictions += (val_predicted == val_target_seqs).sum().item()
                    val_total_predictions += val_target_seqs.numel()

            val_acc = val_correct_predictions / val_total_predictions
            val_loss = val_total_loss / len(val_loader)

            # Log metrics to wandb
            wandb.log({
                "val_loss": val_loss,
                "val_accuracy": val_acc
            })
            
wandb.agent(sweep_id, train_model, count=1)  # Run 1 experiment for hyperparameter tuning


wandb: Agent Starting Run: apff3p2f with config:
wandb: 	bidirectional: True
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	emb_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layer_size: 256
wandb: 	n_dec_layers: 1
wandb: 	n_enc_layers: 3


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2SeqAttention(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 32)
    (rnn): RNN(32, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): AttentionDecoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 32)
    (rnn): RNN(32, 256, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=256, out_features=100, bias=True)
  )
)


Validation: 100%|██████████| 64/64 [00:02<00:00, 24.87it/s]


train_accuracy,▁▆▇██
train_loss,█▂▂▁▁
val_accuracy,▄▁▆▄█
val_loss,▇█▄▅▁
train_accuracy,0.56257
train_loss,2.0281
val_accuracy,0.60308
val_loss,1.89923


wandb: Agent Starting Run: lqk9nfds with config:
wandb: 	bidirectional: True
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	emb_size: 256
wandb: 	epochs: 3
wandb: 	hidden_layer_size: 64
wandb: 	n_dec_layers: 2
wandb: 	n_enc_layers: 2


Seq2SeqAttention(
  (encoder): Encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(100, 256)
    (rnn): GRU(256, 64, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): AttentionDecoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (embedding): Embedding(100, 256)
    (rnn): GRU(256, 64, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=64, out_features=100, bias=True)
  )
)


Training:  68%|██████▊   | 544/800 [00:36<00:16, 15.96it/s]wandb: Ctrl + C detected. Stopping sweep.


In [66]:
# Define the sweep configuration
best_att_sweep = {
    'method': 'bayes',  # Random search
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  # Objective: Maximize accuracy
    },
    'parameters': {
        'cell_type': {'values': ['GRU']},
        'n_enc_layers': {'values': [2]},
        'n_dec_layers': {'values': [2]},
        'hidden_layer_size': {'values': [32]},
        'emb_size': {'values': [256]},
        'bidirectional': {'values': [True]},
        'dropout': {'values': [0.2]},
        'epochs': {'values': [8]} 
    }
}


# Initialize a new wandb sweep
sweep_id = wandb.sweep(best_att_sweep, project="DL_Assignment-3")

Create sweep with ID: d9ay6flq
Sweep URL: https://wandb.ai/lokendrakumar/DL_Assignment-3/sweeps/d9ay6flq


In [1]:
# import torch
# import torch.nn as nn

# def train_model():
#     # Initialize a new wandb run
#     with wandb.init() as run:
#         # Construct run name based on hyperparameters
#         run_name = f"{wandb.config.cell_type}_enc-{wandb.config.n_enc_layers}_dec-{wandb.config.n_dec_layers}_hidden-{wandb.config.hidden_layer_size}_emb-{wandb.config.emb_size}_bi-{wandb.config.bidirectional}_drop-{wandb.config.dropout}_epochs-{wandb.config.epochs}"
#         wandb.run.name = run_name

#         # Initialize the model
#         model = build_model(wandb.config.cell_type,
#                             wandb.config.n_enc_layers,
#                             wandb.config.n_dec_layers,
#                             wandb.config.hidden_layer_size,
#                             num_encoder_tokens,
#                             num_decoder_tokens,
#                             wandb.config.dropout,
#                             wandb.config.emb_size,
#                             device,
#                             wandb.config.bidirectional)
#         model.to(device)

#         # Define the loss function and optimizer
#         criterion = nn.CrossEntropyLoss()  # Use nn.CrossEntropyLoss()
#         optimizer = torch.optim.Adam(model.parameters())

#         # Training and validation loop
#         for epoch in range(wandb.config.epochs):
#             model.train()
#             total_loss = 0
#             correct_predictions = 0
#             total_predictions = 0

#             for input_seqs, target_seqs in tqdm(train_loader, desc="Training"):
#                 input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
#                 optimizer.zero_grad()
#                 output = model(input_seqs, target_seqs)
#                 loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
#                 loss.backward()
#                 optimizer.step()
#                 total_loss += loss.item()
                
#                 _, predicted = torch.max(output, 2)
#                 correct_predictions += (predicted == target_seqs).sum().item()
# #                 print(f"predicted: {predicted} target_seqs:{target_seqs}")
#                 total_predictions += target_seqs.numel()

#             train_acc = correct_predictions / total_predictions
#             train_loss = total_loss / len(train_loader)
#             wandb.log({
#                 "train_loss": train_loss,
#                 "train_accuracy": train_acc,
#             })

#             # Test phase
#             model.eval()
#             test_total_loss = 0
#             test_correct_predictions = 0
#             test_total_predictions = 0
#             with torch.no_grad():
#                 for test_input_seqs, test_target_seqs in tqdm(test_loader, desc="Test"):
#                     test_input_seqs, test_target_seqs = test_input_seqs.to(device), test_target_seqs.to(device)
#                     test_output = model(test_input_seqs, test_target_seqs)
#                     test_loss = criterion(test_output.view(-1, test_output.size(-1)), test_target_seqs.view(-1))
#                     test_total_loss += test_loss.item()
 
#                     _, test_predicted = torch.max(test_output, 2)
#                     test_correct_predictions += (test_predicted == test_target_seqs).sum().item()
#                     test_total_predictions += test_target_seqs.numel()

#             test_acc = test_correct_predictions / test_total_predictions
#             test_loss = test_total_loss / len(test_loader)

#             # Log metrics to wandb
#             wandb.log({
#                 "test_loss": test_loss,
#                 "test_accuracy": test_acc
#             })
            
# wandb.agent(sweep_id, train_model, count=1)  # Run 1 experiment for hyperparameter tuning
